In [6]:
# running python version 3.6

import bs4 as bs
# need to pip install beautifulsoup4
import urllib.request
import wikipedia

In [2]:
source = urllib.request.urlopen('https://www.allrecipes.com/recipe/228293/curry-stand-chicken-tikka-masala-sauce/?internalSource=hub%20recipe&referringContentType=Search').read()

soup = bs.BeautifulSoup(source,'lxml')
# need to pip install lxml

body = soup.body

In [4]:
body

<body data-scoby-impression='{"id": "228293", "eventType": "Allrecipes.Recipe.PageView", "eventCategory": "Page.View", "value": {"user": {"loginStatus":"no","visitorType":"anonymous"}}}' ng-app="allrecipes">
<script>
        var Pubsub = function () {
            "use strict";
            var cache = {};
            var instance = this;
            this.isListening = function (topicName, subscriberName) {
                var ret = false;
                if (cache[topicName]) {
                    ret = cache[topicName].filter(function (item) {
                        return item.name == subscriberName;
                    }).length > 0;
                }
                return ret;
            };

            this.listen = function (topicName, subscriberName, subscribingFunction) {
                if (cache[topicName] == undefined) {
                    cache[topicName] = [];
                }

                cache[topicName].push({ name: subscriberName, func: subscribingFunction });


In [20]:
section = wikipedia.WikipediaPage(title='List of cooking techniques').links

In [21]:
section

['Air fryer',
 'Al dente',
 'Alcohol',
 'Alcoholic beverage',
 'Alkaline',
 'Almond',
 'Amandine (culinary term)',
 'Amylolytic process',
 'Anti-griddle',
 'Anticuchos',
 'Arrosticini',
 'Asado',
 'Aspic',
 'Au gratin',
 'Au jus',
 'Auguste Escoffier',
 'Awadh',
 'Backwoods cooking',
 'Baghaar',
 'Bain-marie',
 'Bakeries',
 'Baking',
 'Bakkwa',
 'Bananas Foster',
 'Barbacoa',
 'Barbecue',
 'Barbecue chicken',
 'Barbecue grill',
 'Barbecue in North Carolina',
 'Barbecue in Texas',
 'Barbecue in Virginia',
 'Barbecue in the United States',
 'Barbecue restaurant',
 'Barbecue sandwich',
 'Barbecue sauce',
 'Barbecuing',
 'Barrel barbecue',
 'Beef',
 'Beef ribs',
 'Better Homes and Gardens (magazine)',
 'Big Pig Jig',
 'Biopreservation',
 'Blackening (cooking)',
 'Bladder',
 'Blanching (cooking)',
 'Boiling',
 'Boiling in cooking',
 'Boiling point',
 'Braising',
 'Brine',
 'Briquette',
 'Broasting',
 'Brochette',
 'Browning (chemical process)',
 'Browning (partial cooking)',
 'Buccan',
 'Bu

In [23]:
for div in body.find_all(class_='recipe-ingred_txt added'):
    print(div.text)

print("next_sec")

test_prep = []

for div in body.find_all(class_='prepTime__item'):
    if div.text != "":
        test_prep.append(div.text)

print(test_prep)

test_directions = []

for div in body.find_all(class_='recipe-directions__list--item'):
    if div.text != "":
        test_directions.append(div.text)

print("Next Test Section")

print(test_directions)

2 tablespoons ghee (clarified butter)
1 onion, finely chopped
4 cloves garlic, minced
1 tablespoon ground cumin
1 teaspoon salt
1 teaspoon ground ginger
1 teaspoon cayenne pepper
1/2 teaspoon ground cinnamon
1/4 teaspoon ground turmeric
1 (14 ounce) can tomato sauce
1 cup heavy whipping cream
2 teaspoons paprika
1 tablespoon white sugar
1 tablespoon vegetable oil
4 skinless, boneless chicken breast halves, cut into bite-size pieces
1/2 teaspoon curry powder
1/2 teaspoon salt, or to taste (optional)
1 teaspoon white sugar, or to taste (optional)
next_sec
['\nPrep15 m\n', '\nCook1 h 5 m\n', '\nReady In1 h 20 m\n']
Next Test Section
['Heat ghee in a large skillet over medium heat and cook and stir onion until translucent, about 5 minutes. Stir in garlic; cook and stir just until fragrant, about 1 minute. Stir cumin, 1 teaspoon salt, ginger, cayenne pepper, cinnamon, and turmeric into the onion mixture; fry until fragrant, about 2 minutes.\n                            ', 'Stir tomato sauce